## Segment Anything Model for Geospatial Data
- Resource: https://samgeo.gishub.org/examples/satellite/#visualize-the-results
- Goal:
  - Use SAM (Segment Anything Model) to create segmentations specifically for buildings from large .tif files.
  - Useful for processing large images that may not fit in memory all at once.

### Step 0: Connect to Google Colab's GPU accelerator.

In [ ]:
# Checking for an available GPU
# Expected output: '/device:GPU:0'
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# Checking for RAM
!cat /proc/meminfo

MemTotal:       13290472 kB
MemFree:         5081492 kB
MemAvailable:   11837368 kB
Buffers:          536860 kB
Cached:          6157828 kB
SwapCached:            0 kB
Active:          1551128 kB
Inactive:        5942592 kB
Active(anon):       2556 kB
Inactive(anon):   812764 kB
Active(file):    1548572 kB
Inactive(file):  5129828 kB
Unevictable:          16 kB
Mlocked:              16 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1632 kB
Writeback:             0 kB
AnonPages:        797084 kB
Mapped:           766468 kB
Shmem:             16280 kB
KReclaimable:     403044 kB
Slab:             481604 kB
SReclaimable:     403044 kB
SUnreclaim:        78560 kB
KernelStack:        5852 kB
PageTables:        10344 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6645236 kB
Committed_AS:    3241428 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       80196 kB
VmallocChunk:    

### Step 1: Install libraries and necessary dependencies.

In [ ]:
%pip install segment-geospatial groundingdino-py leafmap localtileserver

from google.colab import drive
import os, leafmap
from pathlib import Path
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 17.4 MB/s eta 0:00:00
  Created wheel for groundingdino-py: filename=groundingdino_py-0.4.0-py2.py3-none-any.whl size=88735 sha256=ecde70f1a42523a2e377c0a98fe446d3a548648775e5761ce0f0cfae790b0989
  Stored in directory: /root/.cache/pip/wheels/72/25/30/97b491abad279d329c62bef1e91bc56bf2fd40b22281068e1d
Successfully built groundingdino-py


### Step 2: Connect to Google Drive.

In [ ]:
def mount_google_drive():
    drive_mount_path = '/content/drive'
    try:
        if not os.path.ismount(drive_mount_path):
            drive.mount(drive_mount_path)
            print("Google Drive mounted successfully.")
        else:
            print("Google Drive is already mounted.")
    except Exception as e:
        print(f"Failed to mount Google Drive: {e}")


def find_and_change_dir(target_dir, start_path='/'):
    # Try to find the target directory
    for root, dirs, files in os.walk(start_path):
        if target_dir in dirs:
            full_path = os.path.join(root, target_dir)
            print(f"Success: Found the directory at {full_path}")
            os.chdir(full_path)
            print(f"Success: Changed directory to {full_path}")
            return True
    print("Failure: Could not find the target directory.")
    return False

def create_path_to_folder(sub_folder):
    # Check if the sub-folder exists and create a path
    if os.path.exists(sub_folder):
        full_path = os.path.abspath(sub_folder)
        print(f"Success: Found the folder {sub_folder}. Full path is {full_path}")
        return full_path
    else:
        print(f"Failure: The folder {sub_folder} does not exist.")
        return None


# Mount the Drive
mount_google_drive()

# Start by finding and changing directory to CIS4496_EY
if find_and_change_dir('CIS4496_EY'):
    # If successful, create path to given/OtherTeamsData folder
    other_teams_data_path = create_path_to_folder('given/OtherTeamData')
    if other_teams_data_path:
        print(f"Ready for further actions in {other_teams_data_path}")
    else:
        print("Could not proceed to the next step.")
else:
    print("Please ensure the CIS4496_EY directory exists and is accessible from the mounted drive.")

Google Drive is already mounted.
Success: Found the directory at /content/drive/MyDrive/EYProject/CIS4496_EY
Success: Changed directory to /content/drive/MyDrive/EYProject/CIS4496_EY
Success: Found the folder given/OtherTeamData. Full path is /content/drive/MyDrive/EYProject/CIS4496_EY/given/OtherTeamData
Ready for further actions in /content/drive/MyDrive/EYProject/CIS4496_EY/given/OtherTeamData


### Step 3: Create an interactive map.

In [ ]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")

# Allow free zoomings and drawings
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

Map(center=[-22.17615, -51.253043], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [ ]:
# Interactive map demonstration
m

### Step 4: Initialize LangSAM class and TIFF directory.

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

In [ ]:
# Link to TIFF directory
pre_event_tiles = '/content/drive/MyDrive/EYProject/CIS4496_EY/generated_data/Pre_Event_Tiles'

In [ ]:
# Visualize one example TIFF from pre_event_tiles
image = '/content/drive/MyDrive/EYProject/CIS4496_EY/generated_data/Pre_Event_Tiles/pre_random_tile_0.tif'

# Display on the map
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

Map(center=[-22.17615, -51.253043], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

### Step 5: Segment the image and visualize each result.

In [ ]:
# for filename in os.listdir(pre_event_tiles):
#     if filename.endswith('.tif'):
#         # Segment the image
#         sam.predict(image, text_prompt, box_threshold=0.24, text_threshold=0.24)

#         print(f"Processed {filename}")

Processed pre_random_tile_0.tif
Processed pre_random_tile_1.tif
Processed pre_random_tile_2.tif
Processed pre_random_tile_3.tif
Processed pre_random_tile_4.tif
Processed pre_random_tile_5.tif
Processed pre_random_tile_6.tif
Processed pre_random_tile_7.tif
Processed pre_random_tile_8.tif
Processed pre_random_tile_9.tif


In [ ]:
# Segment the image
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

# Polygonize the raster data
# and save as a GeoPackage file
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

# and also save vector data in GeoPandas
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

In [ ]:
# Visualize the result
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m